In [2]:
import pandas as pd

In [3]:
df_inp = pd.read_csv('nse_sensex.csv')

In [4]:
df_inp['DATE'] = pd.to_datetime(df_inp['DATE'])

In [5]:
df_inp.head()

,DATE,SYMBOL,SERIES,PREV CLOSE,OPEN,HIGH,LOW,LAST,CLOSE,VWAP,VOLUME,TURNOVER,TRADES,DELIVERABLE VOLUME,%DELIVERBLE
0,2016-01-01,ASIANPAINT,EQ,883.55,882.0,885.60,876.9,879.7,878.75,880.60,294006,2.589019e+13,7988,149531,0.5086
1,2016-01-04,ASIANPAINT,EQ,878.75,876.2,892.45,871.7,880.5,880.80,881.96,709103,6.254040e+13,28981,337383,0.4758
2,2016-01-05,ASIANPAINT,EQ,880.80,881.3,905.00,881.3,901.4,900.65,898.75,1372748,1.233761e+14,52028,533231,0.3884
3,2016-01-06,ASIANPAINT,EQ,900.65,904.5,911.90,884.0,885.3,886.40,897.11,1716479,1.539873e+14,51374,1008881,0.5878
4,2016-01-07,ASIANPAINT,EQ,886.40,883.0,891.55,869.0,871.1,872.35,878.39,990333,8.698976e+13,26249,477288,0.4819


In [6]:
df_inp.SYMBOL.nunique()

30

In [7]:
df_inp.DATE.agg(['min','max'])

min   2016-01-01
max   2021-06-15
Name: DATE, dtype: datetime64[ns]

In [14]:
## day based features - ordinal data
df_inp['year']=df_inp['DATE'].dt.year.astype('str') 
df_inp['month']=df_inp['DATE'].dt.month.astype('str') 
df_inp['day']=df_inp['DATE'].dt.day.astype('str')

df_inp['dayofweek_num']=df_inp['DATE'].dt.dayofweek.astype('str')  

In [9]:
## create time index for pytorch-forecasting
df_inp = df_inp.sort_values(by=['SYMBOL','DATE'])
df_inp['time_idx'] = df_inp.groupby('SYMBOL').transform('cumcount')
df_inp['time_idx'] = df_inp['time_idx'] + 1

In [10]:
## entity embedding for categoricals

## 

In [11]:
# ## data processing
# start_date = pd.to_datetime('2016-10-01')
# end_date = pd.to_datetime('2018-10-01')
# forecast_steps = 5
# valid_period = 120 # num days
 

# # level of forecast
# level_foreca = 'SYMBOL'
# list_symbols = df_inp[level_foreca].unique().tolist()

# static features = ['SYMBOL','month','day','day_of_week_num']
# dynamic_features = 

In [12]:
df_inp['time_idx'].agg(['min','max'])

min       1
max    1349
Name: time_idx, dtype: int64

In [16]:
from pytorch_forecasting import TimeSeriesDataSet

# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    df_inp,
    group_ids=["SYMBOL"],
    target="CLOSE",
    time_idx="time_idx",
    min_encoder_length=365,
    max_encoder_length=365,
    min_prediction_length=5,
    max_prediction_length=5,
    min_prediction_idx=1000, ## for test/valid set.. 2020 june
    static_categoricals=['SYMBOL'],
    # static_reals=,
    time_varying_known_categoricals=['month','day','dayofweek_num'],
    # time_varying_known_reals=,
    # time_varying_unknown_categoricals=,
    time_varying_unknown_reals=['VOLUME','HIGH','LOW','CLOSE']

)

In [17]:
# convert the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=4)

In [ ]:
from pytorch_forecasting.models.base_model import BaseModelWithCovariates

print(BaseModelWithCovariates.__doc__)


    Model with additional methods using covariates.

    Assumes the following hyperparameters:

    Args:
        static_categoricals (List[str]): names of static categorical variables
        static_reals (List[str]): names of static continuous variables
        time_varying_categoricals_encoder (List[str]): names of categorical variables for encoder
        time_varying_categoricals_decoder (List[str]): names of categorical variables for decoder
        time_varying_reals_encoder (List[str]): names of continuous variables for encoder
        time_varying_reals_decoder (List[str]): names of continuous variables for decoder
        x_reals (List[str]): order of continuous variables in tensor passed to forward function
        x_categoricals (List[str]): order of categorical variables in tensor passed to forward function
        embedding_sizes (Dict[str, Tuple[int, int]]): dictionary mapping categorical variables to tuple of integers
            where the first integer denotes the nu